In [1]:
import io
import numpy as np
import string
import matplotlib.pyplot as plt
import re

In [2]:
x_data = []
y_data = []
text = ""

In [4]:
stopwords = ['are','you','your','an','the','to','for','or', 'is', 'a', 'and', 'have', 'from', 'of', 'with', 'on']

In [5]:
with open('./spam.csv','r') as f:
    for line in f:
        if line[0] == 's':
            y_data.append(1)
            raw_text = line[5:-1]
        else:
            y_data.append(0)
            raw_text = line[4:-1]
        punc = string.punctuation
        nopunc = ""
        for i in range(len(punc)):
            nopunc += " "
        punc2space_table = str.maketrans(punc,nopunc)
        translator = str.maketrans('', '', string.punctuation)
#        nopunc_text = raw_text.translate(table)
        nopunc_text = raw_text.translate(punc2space_table)

        clean_text = str.lower(nopunc_text)
        clean_text = re.sub(' +',' ',clean_text)
        nostopwords_text = ''
        word_text = clean_text.split()
        for word in word_text:
            if not(word in stopwords):
                nostopwords_text = nostopwords_text + word + ' '
        clean_text = nostopwords_text
        x_data.append(clean_text)
        text = text+clean_text

In [6]:
y_data = np.array(y_data)

M = int(len(x_data)*0.7)

x_train = x_data[:M]
y_train = y_data[:M]

x_test = x_data[M:]
y_test = y_data[M:]

In [7]:
word_text = text.split()
vocab = set(word_text)
vocab_to_int = { c : i for i , c in enumerate (vocab)}
int_to_vocab = dict(enumerate(vocab))
V = len(vocab)

In [8]:
spam_prior = np.size(np.nonzero(y_train)[0])/M
mult_probs_spam = np.zeros((V,1))
mult_probs_ham = np.zeros((V,1))

In [9]:
for i in range(M):
    word_text = x_train[i].split()
    n = len(word_text)
    for j in range(n):
        for k in range(V):
            if vocab_to_int[word_text[j]] == k and y_train[i] == 1:
                mult_probs_spam[k] += 1
            elif vocab_to_int[word_text[j]] == k and y_train[i] == 0:
                mult_probs_ham[k] += 1

In [13]:
nsum_spam = 0
nsum_ham = 0

In [14]:
for i in range(M):
    word_text = x_train[i].split()
    n = len(word_text)
    if y_train[i] == 1:
        nsum_spam += n
    else:
        nsum_ham += n

In [15]:
mult_probs_spam = (mult_probs_spam + 1)/(nsum_spam + V)
mult_probs_ham = (mult_probs_ham + 1)/(nsum_ham + V)

In [32]:
y_test.shape

(1674,)

In [33]:
np.squeeze(y_pred).shape

(1673,)

In [42]:
M = len(x_test)
y_test = y_test[:M]
y_pred = np.ones((M,1))
for i in range(M):
    word_text = x_test[i].split()
    n = len(word_text)
    spam_prob = np.log(spam_prior)
    ham_prob = np.log(1 - spam_prior)
    for j in range(n):
        spam_prob +=  np.log(mult_probs_spam[vocab_to_int[word_text[j]]])
        ham_prob += np.log(mult_probs_ham[vocab_to_int[word_text[j]]])
    spam_posterior = np.exp(spam_prob)/(np.exp(spam_prob) + np.exp(ham_prob))
    ham_posterior = 1 - spam_posterior
    if spam_posterior > ham_posterior:
        y_pred[i] = 0

test_acc = np.size(np.nonzero(y_test[:,None] == y_pred)[0])/M
print('test accuracy = %f'%(test_acc))

test accuracy = 0.858338


C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':
C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in greater
  
